## Import et configurations MlFlows

In [ ]:
import mlflow
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

MLFLOW_TRACKING_URI = "http://localhost:5555"  
EXPERIMENT_NAME = "stockout_substitution_hyperopt_classifier_ranker_5"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    raise ValueError(f"Expérience '{EXPERIMENT_NAME}' introuvable")

EXPERIMENT_ID = experiment.experiment_id
print("Experiment ID:", EXPERIMENT_ID)


Experiment ID: 6


## Chargement de tous les runs

In [2]:
runs = mlflow.search_runs(
    experiment_ids=[EXPERIMENT_ID],
    output_format="pandas"
)

print(f"{len(runs)} runs chargés")
runs.head(3)


100 runs chargés


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.hit_rate_at_3,metrics.hit_rate_at_5,metrics.hit_rate_at_1,metrics.ndcg_at_5,metrics.ndcg_at_3,metrics.best_iteration,metrics.ndcg_at_1,metrics.precision,metrics.logloss,metrics.auc,metrics.recall,metrics.pr_auc,metrics.f1,params.subsample,params.min_child_samples,params.colsample_bytree,params.n_estimators,params.learning_rate,params.num_leaves,params.depth,params.l2_leaf_reg,params.rsm,params.iterations,params.reg_lambda,params.reg_alpha,params.max_depth,params.min_child_weight,params.C,params.penalty,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type
0,0b8f4a7c9cc34228bbd056e0bab0d93a,6,FINISHED,mlflow-artifacts:/6/0b8f4a7c9cc34228bbd056e0ba...,2026-01-07 02:18:40.698000+00:00,2026-01-07 02:18:43.909000+00:00,0.712296,0.715670,0.636238,0.886897,0.855774,4.0,0.636238,NaN,NaN,NaN,NaN,NaN,NaN,0.7,100,1.0,500,0.1,63,None,None,None,None,None,None,None,None,None,None,resilient-lamb-873,eric,/home/eric/.cache/pypoetry/virtualenvs/algo-re...,LOCAL
1,99e3296c7431475c834a6ddd7fc4cfa7,6,FINISHED,mlflow-artifacts:/6/99e3296c7431475c834a6ddd7f...,2026-01-07 02:18:37.477000+00:00,2026-01-07 02:18:40.580000+00:00,0.712296,0.715757,0.637536,0.887357,0.856644,17.0,0.637536,NaN,NaN,NaN,NaN,NaN,NaN,1.0,100,0.7,500,0.05,31,None,None,None,None,None,None,None,None,None,None,loud-bear-269,eric,/home/eric/.cache/pypoetry/virtualenvs/algo-re...,LOCAL
2,dfd496cff2114b3b88dbc565c1d07b50,6,FINISHED,mlflow-artifacts:/6/dfd496cff2114b3b88dbc565c1...,2026-01-07 02:18:34.135000+00:00,2026-01-07 02:18:37.374000+00:00,0.712296,0.715670,0.638141,0.887685,0.857083,15.0,0.638141,NaN,NaN,NaN,NaN,NaN,NaN,0.9,20,0.9,1000,0.1,31,None,None,None,None,None,None,None,None,None,None,peaceful-mole-911,eric,/home/eric/.cache/pypoetry/virtualenvs/algo-re...,LOCAL


## Colonnes utiles et nettoyage

In [3]:
metric_cols = [c for c in runs.columns if c.startswith("metrics.")]
param_cols = [c for c in runs.columns if c.startswith("params.")]
tag_cols   = [c for c in runs.columns if c.startswith("tags.")]

cols = (
    ["run_id", "status", "start_time"] +
    metric_cols +
    param_cols +
    tag_cols
)

df = runs[cols].copy()

# Simplification des noms
df.columns = (
    df.columns
      .str.replace("metrics.", "", regex=False)
      .str.replace("params.", "", regex=False)
      .str.replace("tags.", "", regex=False)
)

df.head(2)


,run_id,status,start_time,hit_rate_at_3,hit_rate_at_5,hit_rate_at_1,ndcg_at_5,ndcg_at_3,best_iteration,ndcg_at_1,precision,logloss,auc,recall,pr_auc,f1,subsample,min_child_samples,colsample_bytree,n_estimators,learning_rate,num_leaves,depth,l2_leaf_reg,rsm,iterations,reg_lambda,reg_alpha,max_depth,min_child_weight,C,penalty,mlflow.runName,mlflow.user,mlflow.source.name,mlflow.source.type
0,0b8f4a7c9cc34228bbd056e0bab0d93a,FINISHED,2026-01-07 02:18:40.698000+00:00,0.712296,0.715670,0.636238,0.886897,0.855774,4.0,0.636238,NaN,NaN,NaN,NaN,NaN,NaN,0.7,100,1.0,500,0.1,63,None,None,None,None,None,None,None,None,None,None,resilient-lamb-873,eric,/home/eric/.cache/pypoetry/virtualenvs/algo-re...,LOCAL
1,99e3296c7431475c834a6ddd7fc4cfa7,FINISHED,2026-01-07 02:18:37.477000+00:00,0.712296,0.715757,0.637536,0.887357,0.856644,17.0,0.637536,NaN,NaN,NaN,NaN,NaN,NaN,1.0,100,0.7,500,0.05,31,None,None,None,None,None,None,None,None,None,None,loud-bear-269,eric,/home/eric/.cache/pypoetry/virtualenvs/algo-re...,LOCAL


## Separation Ranker et Classifier

In [4]:
df["model_type"] = df.get("model_type", "unknown")
df["model_name"] = df.get("model_name", "unknown")

df_classif = df[df["model_type"] == "classification"].copy()
df_ranker  = df[df["model_type"] == "ranking"].copy()

print("Classifiers:", len(df_classif))
print("Rankers:", len(df_ranker))


Classifiers: 0
Rankers: 0


## Analyse Classifier

Métriques disponibles:
- auc
- pr_auc
- logloss
- precision
- recall
- f1

#### Top modèles par AUC

In [ ]:
df_classif_sorted = (
    df_classif
    .sort_values("auc", ascending=False)
)

df_classif_sorted[
    ["model_name", "auc", "pr_auc", "f1", "precision", "recall", "logloss"]
].head(10)


#### Moyenne performance par modèle (classifier)

In [ ]:
classif_summary = (
    df_classif
    .groupby("model_name")[["auc", "pr_auc", "f1", "precision", "recall", "logloss"]]
    .mean()
    .sort_values("auc", ascending=False)
)

classif_summary


#### Meilleur run par modèle

In [ ]:
best_classif_runs = (
    df_classif
    .sort_values("auc", ascending=False)
    .groupby("model_name")
    .head(1)
)

best_classif_runs[
    ["model_name", "auc", "pr_auc", "f1", "precision", "recall", "logloss"]
]


## Analyse des Rankers

Métriques disponibles:
- ndcg_at_1
- ndcg_at_3
- ndcg_at_5
- hit_rate_at_1
- hit_rate_at_3
- hit_rate_at_5

#### Top runs par NDCG@3

In [ ]:
df_ranker_sorted = df_ranker.sort_values("ndcg_at_3", ascending=False)

df_ranker_sorted[
    ["model_name", "ndcg_at_1", "ndcg_at_3", "ndcg_at_5",
     "hit_rate_at_1", "hit_rate_at_3", "hit_rate_at_5"]
].head(10)


#### Moyenne des performances Ranker

In [ ]:
ranker_summary = (
    df_ranker
    .groupby("model_name")[
        ["ndcg_at_1", "ndcg_at_3", "ndcg_at_5",
         "hit_rate_at_1", "hit_rate_at_3", "hit_rate_at_5"]
    ]
    .mean()
    .sort_values("ndcg_at_3", ascending=False)
)

ranker_summary


#### Meilleur run par Ranker

In [ ]:
best_ranker_runs = (
    df_ranker
    .sort_values("ndcg_at_3", ascending=False)
    .groupby("model_name")
    .head(1)
)

best_ranker_runs[
    ["model_name", "ndcg_at_1", "ndcg_at_3", "ndcg_at_5",
     "hit_rate_at_1", "hit_rate_at_3", "hit_rate_at_5"]
]


#### Inspection des hyperparamètres du meilleur run

In [ ]:
best_run = best_ranker_runs.iloc[0]

best_params = best_run[[c for c in best_run.index if c not in [
    "run_id","model_name","model_type","status","start_time"
] and not pd.isna(best_run[c])]]

best_params


### Conclusions

- Le meilleur classifier selon l'AUC est : **XXX**
- Le meilleur ranker selon NDCG@3 est : **XXX**
- Les Rankers surpassent les classifiers en top-k → **à privilégier en prod**
- Les classifiers restent utiles comme baseline / fallback
